In [8]:
%run imports.ipynb
%run information_conditions.ipynb import Information_Conditions
%run base_ecopg.ipynb import BaseEcologicalPublicGood
%run helper_functions.ipynb import *
%run simulation_and_results_functions.ipynb import *

In [ ]:

def compare_conditions_cooperation_basin_size(num_samples= 100, degraded_choice = False, m_value = -6, discount_factor = 0.98, exclude_degraded_state_for_average_cooperation = True , information_modes = all_information_modes):
    """
    Runs simulations for different information conditions and outputs 
    the results for each condition.
    
    Parameters:
        ecopg (EcologicalPublicGood): An instance of the ecological public good model.
        num_samples (int): Number of initial conditions to sample.
        Tmax (int): Maximum time steps for trajectory simulation.
        tolerance (float): Convergence tolerance for fixed point detection.
        
    Returns:
        None (prints the output summaries for each information condition)
    """

    print(locals())
    
    basin_of_attraction_cooperation_results = {}
    
    
    ecopg = BaseEcologicalPublicGood(m = m_value, degraded_choice=degraded_choice)

    for mode in information_modes:
        # Initialize the information condition
        information_condition_instance = Information_Conditions(ecopg, mode=mode)
        mae = POstratAC_eps(env=information_condition_instance, learning_rates=0.01, discount_factors= discount_factor)

        # Data storage

        # print(f"\nMode: {mode}")

        avg_coop_time_pairs = run_simulation_across_conditions(
            mae = mae, 
            mode = mode,
            num_samples = num_samples, 
            exclude_degraded_state_for_average_cooperation = exclude_degraded_state_for_average_cooperation
        )

        cooperation_basin_size = get_results_only_cooperation_basin_of_attraction_size(avg_coop_time_pairs)

        basin_of_attraction_cooperation_results[mode] = cooperation_basin_size


    return basin_of_attraction_cooperation_results
    

# Example usage:
data = compare_conditions_cooperation_basin_size(degraded_choice=False)


{'num_samples': 50, 'degraded_choice': False, 'm_value': -6, 'discount_factor': 0.98, 'exclude_degraded_state_for_average_cooperation': True, 'information_modes': ['both_state_and_action_information', 'only_action_history_information', 'only_state_information', 'no_information']}


In [ ]:
data = compare_conditions_cooperation_basin_size(degraded_choice=False)
# print(data_test)

{'num_samples': 1000, 'degraded_choice': False, 'm_value': -6, 'discount_factor': 0.98, 'exclude_degraded_state_for_average_cooperation': True, 'information_modes': ['both_state_and_action_information', 'only_action_history_information', 'only_state_information', 'no_information']}
  [0.201 0.799]
  [0.971 0.029]
  [0.766 0.234]]

 [[0.102 0.898]
  [0.93  0.07 ]
  [0.017 0.983]
  [0.559 0.441]]] only_action_history_information
  [0.581 0.419]
  [1.    0.   ]
  [0.834 0.166]]

 [[0.775 0.225]
  [0.26  0.74 ]
  [0.98  0.02 ]
  [0.522 0.478]]] only_action_history_information
  [0.421 0.579]
  [0.98  0.02 ]
  [0.193 0.807]]

 [[0.864 0.136]
  [0.968 0.032]
  [0.916 0.084]
  [0.412 0.588]]] only_action_history_information
  [0.323 0.677]
  [0.639 0.361]
  [0.046 0.954]]

 [[0.607 0.393]
  [0.986 0.014]
  [0.03  0.97 ]
  [0.161 0.839]]] only_action_history_information
  [0.277 0.723]
  [0.949 0.051]
  [0.027 0.973]]

 [[0.676 0.324]
  [0.828 0.172]
  [0.192 0.808]
  [0.988 0.012]]] only_acti

In [ ]:
# np.save('data_500_samples_0.9_0.1.npy', data)

import pickle

# Your dictionary

# Save to file
with open('data_500_samples_0.9_0.1', 'wb') as f:
    pickle.dump(data, f)


In [ ]:
print(data)


{'both_state_and_action_information': np.float64(65.0), 'only_action_history_information': np.float64(3.8), 'only_state_information': np.float64(66.6), 'no_information': 0}


In [ ]:

# Extract cooperation percentages correctly from the DataFrame

# Define the conditions in order


# Extract cooperation percentages
cooperation_basin_size = [(data[condition]) for condition in all_information_modes]

# Debugging output
print("Extracted Cooperation Basin Size:", cooperation_basin_size)

conditions = [
    "Both Social and Ecological State Information", 
    "Only Social Information", 
    "Only Ecological State Information", 
    "No Information"
]

# Create DataFrame for plotting
plot_df = pd.DataFrame({
    'Information Condition': conditions,
    'Cooperation Basin Size': cooperation_basin_size
})

# Define a color palette
color_map = {
    "Both Social and Ecological State Information": "#4c72b0",  # Muted Blue
    "Only Social Information": "#FFB6C1",  # Muted Pink
    "Only Ecological State Information": "#55a868",  # Muted Green
    "No Information": "#000000"  # Black
}

# Create figure
fig = go.Figure()

for i, row in plot_df.iterrows():
    condition = row['Information Condition']
    percentage = row['Cooperation Basin Size']
    color = color_map[condition]
    
    if percentage == 0:
        # 1. Actual outline bars for zero values (shown in the plot)
        fig.add_trace(go.Bar(
            x=[condition], 
            y=[percentage], 
            marker=dict(color='rgba(0,0,0,0)', line=dict(color=color, width=4)),
            text=f"{float(percentage):.1f}%",
            textposition='outside',
            textfont=dict(size=15, color='black'),  # Larger, darker percentage text
            showlegend=False,  # Don't show this in the legend
            legendgroup=condition  # Group legend with the solid bar
        ))

        # 2. Hidden solid legend bar (only for legend display)
        fig.add_trace(go.Bar(
            x=[None],  # Invisible bar in the plot
            y=[None],
            name=condition,
            marker=dict(color=color),  # Filled marker for the legend
            legendgroup=condition  # Matches legend with outline bar
        ))

    else:
        # Normal filled bars
        fig.add_trace(go.Bar(
            x=[condition], 
            y=[percentage], 
            name=condition,
            marker=dict(color=color),
            text=f"{float(percentage):.1f}%",
            textposition='outside',
            textfont=dict(size=15, color='black')  # Larger, darker percentage text
        ))

# Update layout for aesthetics
fig.update_layout(
    yaxis_title="Cooperation Basin Size (%)",
    yaxis=dict(
        range=[0, 100],
        tickfont=dict(size=18)  # Larger, darker y-axis label
    ),
    xaxis=dict(title='', showticklabels=False),
    plot_bgcolor='snow',  # Clean background
    width=500,
    height=675,
    bargap=0,  # Minimize gaps
    legend=dict(
        title="",
        orientation="h",
        yanchor="bottom",
        y=-0.3,
        xanchor="center",
        x=0.5,
    font=dict(size=13.5, color='black')
    )
)

# Show figure
fig.show()




Extracted Cooperation Basin Size: [np.float64(65.0), np.float64(3.8), np.float64(66.6), 0]
